In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit

data = pd.read_csv("./datasets./Tech_Use_Stress_Wellness.csv")

In [ ]:
print(data)

      user_id  age  gender  daily_screen_time_hours  phone_usage_hours  \
0           1   53    Male                      6.8                2.9   
1           2   66  Female                      4.1                2.1   
2           3   43    Male                      4.7                3.6   
3           4   29  Female                      6.0                4.5   
4           5   57    Male                      6.7                3.4   
...       ...  ...     ...                      ...                ...   
4995     4996   17  Female                      7.0                2.2   
4996     4997   42  Female                      1.9                1.5   
4997     4998   20  Female                      6.3                1.5   
4998     4999   58    Male                      6.2                3.5   
4999     5000   58  Female                      4.8                1.7   

      laptop_usage_hours  tablet_usage_hours  tv_usage_hours  \
0                    2.1                 0.5   

In [ ]:
stress_level = data['stress_level']
mental_score = data['mental_health_score']

rs = ShuffleSplit(n_splits=2, test_size=.7, random_state=0)

dtrain = xgb.DMatrix(x.value)

In [ ]:
dtrain = xgb.DMatrix(data, label=label)